In [1]:
import cv2


In [2]:
img = cv2.imread('C:/Users/adham/Downloads/adham.jpg',1) 
img        

array([[[236, 201, 215],
        [234, 199, 213],
        [237, 202, 216],
        ...,
        [183, 172, 188],
        [179, 167, 185],
        [185, 173, 191]],

       [[232, 197, 211],
        [230, 195, 209],
        [234, 199, 213],
        ...,
        [181, 170, 186],
        [179, 167, 185],
        [179, 167, 185]],

       [[233, 199, 210],
        [231, 197, 208],
        [236, 202, 213],
        ...,
        [187, 177, 190],
        [190, 179, 195],
        [185, 174, 190]],

       ...,

       [[159, 158, 168],
        [159, 158, 168],
        [161, 157, 168],
        ...,
        [115, 113, 142],
        [115, 116, 144],
        [117, 118, 146]],

       [[159, 158, 168],
        [159, 158, 168],
        [161, 157, 168],
        ...,
        [111, 109, 138],
        [113, 111, 141],
        [114, 112, 142]],

       [[158, 157, 167],
        [158, 157, 167],
        [161, 157, 168],
        ...,
        [114, 112, 141],
        [115, 113, 143],
        [116, 114, 144]]

In [3]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [13]:
import os
import cv2
import face_recognition

# Folder with images to process
folder_path = 'C:/Users/adham/Downloads/images/'

# Load the reference image
reference_image_path = folder_path+'adham.jpg'
reference_image = face_recognition.load_image_file(reference_image_path)
reference_face_encoding = face_recognition.face_encodings(reference_image)[0]

# Iterate over all images in the specified folder
for image_name in os.listdir(folder_path):
    if not image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        continue  # Skip non-image files

    image_path = os.path.join(folder_path, image_name)
    img = cv2.imread(image_path)

    # Convert the image to grayscale for face detection
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect faces
    face_locations = face_recognition.face_locations(img)
    face_encodings = face_recognition.face_encodings(img, face_locations)

    # Draw rectangles around detected faces and compare with the reference face
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # Compare this face with the reference face
        matches = face_recognition.compare_faces([reference_face_encoding], face_encoding, tolerance=0.6)

        if True in matches:
            print(f"Match found in image: {image_name}")
            cv2.rectangle(img, (left, top), (right, bottom), (0, 255, 0), 2)

    cv2.imshow('Image with Detected Faces', img)
    k = cv2.waitKey(0)

    if k == 27:
        break  # Exit on pressing 'ESC'

cv2.destroyAllWindows()


Match found in image: adham.jpg
Match found in image: domz.jpg


In [14]:
import os
import cv2
import face_recognition

# Folder with images to process
folder_path = 'C:/Users/adham/Downloads/images/'

# Load all images, their face encodings, and face locations from the folder
image_encodings = []
image_names = []
image_locations = []

# Load images and get face encodings and face locations
for image_name in os.listdir(folder_path):
    if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(folder_path, image_name)
        image = face_recognition.load_image_file(image_path)
        
        # Get face encodings and face locations for each image
        encodings = face_recognition.face_encodings(image)
        locations = face_recognition.face_locations(image)

        if encodings and locations:  # If valid face encodings and locations
            image_encodings.append(encodings[0])
            image_names.append(image_name)
            image_locations.append(locations[0])  # Store the first face location

# Compare each image with all other images
for i, (img_name, img_encoding, img_location) in enumerate(zip(image_names, image_encodings, image_locations)):
    img_path = os.path.join(folder_path, img_name)
    img = cv2.imread(img_path)

    # Draw rectangle around the detected face in the current image
    top, right, bottom, left = img_location
    cv2.rectangle(img, (left, top), (right, bottom), (0, 255, 0), 2)  # Rectangle for visualization

    # Loop through all other images for comparison
    for j, (other_name, other_encoding, other_location) in enumerate(zip(image_names, image_encodings, image_locations)):
        if i == j:
            continue  # Don't compare with the same image

        # Compare face encodings
        matches = face_recognition.compare_faces([other_encoding], img_encoding, tolerance=0.6)

        if True in matches:
            print(f"Match found between {img_name} and {other_name}")

            # Draw rectangle around the matching face in the other image
            top, right, bottom, left = other_location
            cv2.rectangle(img, (left, top), (right, bottom), (0, 255, 0), 2)

    cv2.imshow(f'Image with Matches: {img_name}', img)
    k = cv2.waitKey(0)

    if k == 27:
        break  # Exit on pressing 'ESC'

cv2.destroyAllWindows()


Match found between adham.jpg and domz.jpg
Match found between domz.jpg and adham.jpg
Match found between elonmusk.jpg and eloon.jpg
Match found between eloon.jpg and elonmusk.jpg


In [15]:
import os
import cv2
import face_recognition

# Folder with reference images to process
folder_path = 'C:/Users/adham/Downloads/images/'

# Load face encodings and corresponding names from the folder
reference_face_encodings = []
reference_face_names = []

for image_name in os.listdir(folder_path):
    if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(folder_path, image_name)
        image = face_recognition.load_image_file(image_path)
        
        # Get the face encodings
        encodings = face_recognition.face_encodings(image)
        
        if encodings:  # If there are valid face encodings
            reference_face_encodings.append(encodings[0])
            reference_face_names.append(image_name)  # Using image name to represent the reference name

# Open the front camera (webcam)
camera = cv2.VideoCapture(0)  # 0 typically refers to the default webcam

# Continuously capture frames from the webcam
while True:
    ret, frame = camera.read()
    
    if not ret:
        break  # If capturing failed, break the loop
    
    # Detect faces in the current frame
    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)

    # Compare with reference face encodings
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        matches = face_recognition.compare_faces(reference_face_encodings, face_encoding, tolerance=0.6)
        name = "Unknown"

        if True in matches:
            match_index = matches.index(True)
            name = reference_face_names[match_index]  # Get the name of the matched face

        # Draw a rectangle around the detected face and add the name
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        if name != "Unknown":
            print(f"Match found: {name}")

    cv2.imshow('Webcam Face Detection', frame)
    
    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

camera.release()
cv2.destroyAllWindows()


Match found: adham.jpg
Match found: adham.jpg
Match found: adham.jpg
Match found: adham.jpg
Match found: adham.jpg
Match found: adham.jpg
Match found: adham.jpg
Match found: adham.jpg
Match found: adham.jpg
Match found: adham.jpg
Match found: adham.jpg
Match found: adham.jpg
Match found: adham.jpg
Match found: adham.jpg
Match found: adham.jpg
Match found: adham.jpg
Match found: adham.jpg
Match found: adham.jpg
Match found: adham.jpg
Match found: adham.jpg
Match found: adham.jpg
Match found: adham.jpg
Match found: adham.jpg
Match found: adham.jpg
Match found: adham.jpg
Match found: adham.jpg
Match found: adham.jpg
Match found: adham.jpg
Match found: elonmusk.jpg
Match found: adham.jpg
Match found: elonmusk.jpg
Match found: adham.jpg
Match found: elonmusk.jpg
Match found: adham.jpg
Match found: adham.jpg
Match found: adham.jpg
Match found: adham.jpg
Match found: adham.jpg
Match found: adham.jpg
Match found: adham.jpg
Match found: adham.jpg
Match found: adham.jpg
Match found: adham.jpg
Ma